# Setup

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from utilities import *
from config import *
from dataloading import *
from tqdm import tqdm
from transformer import *
import os

In [ ]:
(
        train_inputs, train_perms, train_dataloader, 
        val_seqs, val_perms, val_dataloader,
        test_seqs, test_perms, test_dataloader,
        dataset_size
) = load_data()

## Load model (accelerator version)

In [ ]:
from accelerate import Accelerator, load_checkpoint_and_dispatch

# setup the model
model = BigramLanguageModel()

accelerator = Accelerator()

# optionally: load the model
save_directory = f"{PATH}/model/{MODELNAME}"
file_path = f"{save_directory}/model.safetensors"

model = load_checkpoint_and_dispatch(model, file_path)

In [ ]:
model

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

## Load model (legacy version)

In [ ]:
from accelerate import Accelerator, load_checkpoint_and_dispatch

# setup the model
model = BigramLanguageModel()

device = "cpu"
model = nn.DataParallel(model)
model = model.to(device)

# optionally: load the model
filename = f"{PATH}/model/{MODELNAME}.pth"

if os.path.isfile(filename):
    model.load_state_dict(torch.load(filename, map_location=torch.device(device)))

os.path.isfile(filename)

In [ ]:
19_225_350

In [ ]:
!pwd

In [ ]:
torch.norm(model.module.position_embedding(torch.arange(block_size)), dim=1)

In [ ]:
INPUT_LENGTH

In [ ]:
import plotly.express as px
from math import sin, cos, tau

# generate embedding
embedding = []

period = 10
N = 2*period

for x in range(N):
    embedding.append(np.array([cos((tau*x)/period), sin((tau*x)/period)]))

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
    similarity.append(row)

figure = px.imshow(similarity, title=f"Periodic embedding similarity matrix")
figure

In [ ]:
import plotly.express as px

embedding = model.module.position_embedding(torch.arange(block_size)).detach().numpy()

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
    similarity.append(row)

similarity = np.array(similarity)

figure = px.imshow(
    -similarity,
    title=f"{MODELNAME} position embedding similarity matrix",
    color_continuous_scale="viridis"
)

hide_ui = True

if hide_ui:
    figure.update_layout(coloraxis_showscale=False)
    figure.update_xaxes(showticklabels=False)
    figure.update_yaxes(showticklabels=False)
    figure.update_layout(
        title="",
        autosize=False,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0
        ),
        width=1000,
        height=1000
    )

figure

In [ ]:
test_perms.apppend(list(range(16)))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

embedding = model.module.position_embedding(torch.arange(block_size)).numpy()

pca = PCA(n_components=37)
pca_reduced = pca.fit_transform(embedding)

tsne = TSNE()
tsne_reduced = tsne.fit_transform(pca_reduced)

In [ ]:
tsne_reduced

In [ ]:
import plotly.express as px

px.scatter(
    x=tsne_reduced[:,0], 
    y=tsne_reduced[:,1], 
    color=["input" for x in range(20)] + ["seperator"] + ["output" for x in range(16)]
)

In [ ]:
import wandb

wandb.login()
run = wandb.init()

In [ ]:
run.log({"image": wandb.Image(np.array(similarity))})

In [ ]:
plausible = 0

for perm in tqdm(test_perms):
    correct = 0

    for pos, char in enumerate(perm):
        if char == pos:
            correct += 1
    
    if correct >= 6:
        plausible += 1

plausible/len(test_perms)

In [ ]:
for x in test_dataloader:
    break

model(x[0][0].reshape((1,-1)))

In [ ]:
model(x[0][0].reshape((1,-1))).argsort(dim=1, descending=True)[0]

In [ ]:
test_perms[0]

In [ ]:
a.argmax()

In [ ]:
15+num_trans

In [ ]:
best = float("inf")

for perm in test_perms:
    fixed = 0

    for pos, char in enumerate(perm):
        fixed += pos == char
    
    best = min(fixed, best)

best

In [ ]:
import plotly.express as px

embedding = model.module.token_embedding_table(torch.arange(vocab_size)).detach().cpu().numpy()

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
    similarity.append(row)

similarity = np.array(similarity)

figure = px.imshow(
    similarity,
    title=f"{MODELNAME} token embedding similarity matrix",
    color_continuous_scale="hot",
    zmin=similarity.min()-0.05
)

hide_ui = True

if hide_ui:
    figure.update_layout(coloraxis_showscale=False)
    figure.update_xaxes(showticklabels=False)
    figure.update_yaxes(showticklabels=False)
    figure.update_layout(
        title="",
        autosize=False,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0
        ),
        width=1000,
        height=1000
    )

figure

In [ ]:
with open(f"./embedding_pictures/position/{MODELNAME}.npy", "wb") as file:
    np.save(file, similarity)

In [ ]:
with open(f"./embedding_pictures/position/{MODELNAME}.npy", "rb") as file:
    b = np.load(file)

b.shape

This next plot is found by taking the embeddings of $s_{i,j}$ at different positions and dotting them together (normed)

In [ ]:
def binarise(pos1, pos2, d=4):
    return torch.tensor([int(x) for x in format(2**(d)*pos1+pos2, f'#0{2*d+2}b')[2:]])

def process_tensor(tensor):
    return tensor.detach().numpy().reshape(-1)

similarity = []

digits = 5

for swap1 in range(2**(digits-1)):
    swap1_slice = []

    for swap2 in range(2**(digits-1)):
        swap2_slice = []

        for x in range(10):
            row = []

            for y in range(10):
                token_embedding = process_tensor(
                    model.module.token_embedding_table(binarise(swap1, swap2, digits))
                )

                vectors = []

                for j in (x, y):
                    position_embedding = process_tensor(
                        model.module.position_embedding(torch.arange(j*digits, (j+2)*digits))
                    )

                    swap_embedding = token_embedding + position_embedding
                    vectors.append(swap_embedding / np.linalg.norm(swap_embedding))
                
                row.append(np.dot(vectors[0], vectors[1]))
            swap2_slice.append(row)
        swap1_slice.append(swap2_slice)
    similarity.append(swap1_slice)

similarity = np.array(similarity)

In [ ]:
import plotly.graph_objects as go

# the following code is chatgpt generated (mostly)

def frame_name(i, j):
    return f"swap_{i}_{j}"

# Create initial heatmap (first slice)
initial_data = similarity[0, 0]
fig = go.Figure(data=go.Heatmap(z=initial_data))

zmin = np.min(similarity)
zmax = np.max(similarity)

# Define frames
frames = [
    go.Frame(
        data=go.Heatmap(z=similarity[i, j], zmin=zmin, zmax=zmax), 
        name=frame_name(i, j)
    )
    for i in range(similarity.shape[0])
    for j in range(similarity.shape[1])
]

# Add frames to the figure
fig.frames = frames

# Create slider steps for both sliders
slider1_steps = [
    {
        "args": [
            [frame_name(i, 0)],
            {
                "frame": {"duration": 300, "redraw": True},
                "mode": "immediate",
                "transition": {"duration": 300},
            },
        ],
        "label": f"Index {i}",
        "method": "animate",
    }
    for i in range(similarity.shape[0])
]

slider2_steps = [
    {
        "args": [
            [frame_name(0, j)],
            {
                "frame": {"duration": 300, "redraw": True},
                "mode": "immediate",
                "transition": {"duration": 300},
            },
        ],
        "label": f"Index {j}",
        "method": "animate",
    }
    for j in range(similarity.shape[1])
]

# Update layout with sliders
fig.update_layout(
    sliders=[
        {
            "currentvalue": {
                "font": {"size": 10},
                "prefix": "Tranposition 1: ",
                "visible": True,
                "xanchor": "center",
            },
            "steps": slider1_steps,
            "yanchor": "top",
            "xanchor": "left",
            "y": -0.1
        },
        {
            "currentvalue": {
                "font": {"size": 10},
                "prefix": "Transposition 2: ",
                "visible": True,
                "xanchor": "center",
            },
            "steps": slider2_steps,
            "yanchor": "top",
            "xanchor": "left",
            "y": -0.2
        }
    ],
    xaxis=dict(
        scaleanchor="y",
        scaleratio=1,
        constrain="domain"
    ),
    yaxis=dict(
        scaleanchor="x",
        scaleratio=1,
        constrain="domain"
    )
)

# Show the figure
fig.show()


In [ ]:
similarity = []

digits = 5

for swap in range(2**digits):
    swap_slice = []

    for kindex in range(MAX_TRANS_NUMBER):

        token_embedding = model.module.token_embedding_table(binarise(swap, swap+1, digits)).detach().numpy()

        matrices = []

        for j in (1, kindex):
            position_embedding = model.module.position_embedding(torch.arange(j*digits, (j+2)*digits)).detach().numpy()

            swap_embedding = token_embedding + position_embedding
            matrices.append(swap_embedding)

        swap_slice.append(np.dot(matrices[0], np.transpose(matrices[1])))
    
    similarity.append(swap_slice)

similarity = np.array(similarity)

In [ ]:
swap+1

In [ ]:
from plotly import optional_imports
nbformat = optional_imports.get_module("nbformat")
print(nbformat)

In [ ]:
import pandas as pd

results = pd.read_csv("./results/window-7.0.csv")

In [ ]:
num_results = results["results"].to_numpy().astype(int)

successes = []
failures = []

for perm, result in zip(test_perms, num_results):
    if result:
        successes.append(perm)
    else:
        failures.append(perm)

In [ ]:
results.head(10)

In [ ]:
val_perms[0]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)

fit = pca.fit_transform(model.module.position_embedding(torch.arange(block_size)).detach().numpy())
fit

In [ ]:
pca.explained_variance_ratio_

In [ ]:
val_seqs[0]

## Custom ouput tests

In [ ]:
def get_perm_hybrid(seq, size):
    og = list(range(size))

    swaps = []

    for pos, x in enumerate(seq):
        if pos % 2 == 0:
            swaps.append([x])
        else:
            swaps[-1].append(x)
    
    for x, y in swaps:
        og[x], og[y] = og[y], og[x]

    return np.array(og)

def convert_to_hybrid(seq, size):
    new = []
    
    for char in seq:
        new.append(int(char//size))
        new.append(int(char%size))
    
    return new

def convert_to_general(seq, size):
    new = []

    for x, y in zip(seq[::2], seq[1::2]):
        new.append(x*size + y)

    return new

custom = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 15]

true = get_perm_hybrid(
    custom, 
    16
)

genned = model.module.generate(
    convert_to_general(custom, 16), 
    force_valid=False, 
    debug=False
)

print("True: ", true)
print("Model:", genned)
print("Same:", (true == genned).all())
print("Sortd:", np.sort(genned))

In [ ]:
a = range(20)

for x, y in zip(a[::2], a[1::2]):
    print(x, y)

In [ ]:
convert_to_general(custom, 16)

In [ ]:
[0 for x in range(20)]

In [ ]:
# wrong
b = torch.tensor([-265.7672, -265.7672, -265.7672, -265.7672, -265.7672, -265.7672,
         -265.7672, -265.7672, -265.7672, -265.7672, -265.7672, -265.7672,
         -265.7672, -265.7672, -265.7672, -265.7672, -240.0325, -247.6589,
         -246.5491, -257.4102, -257.1504, -257.8494, -255.2135, -254.7709,
         -253.9813, -256.4985, -256.2157, -261.1152, -266.6082, -258.3374,
         -273.0349, -261.6855, -265.7672, -265.7672])

import plotly.express as px
px.bar(b[16:32]-b[16:32].max()-1)

In [ ]:
train_perms[0]

In [ ]:
success_fixed_count = []

for perm in successes:
    fixed_count = 0

    for pos, char in enumerate(perm):
        if pos == char:
            fixed_count += 1
    
    success_fixed_count.append(fixed_count)

sum(success_fixed_count)/len(success_fixed_count)

In [ ]:
failure_fixed_count = []

for perm in failures:
    fixed_count = 0

    for pos, char in enumerate(perm):
        if pos == char:
            fixed_count += 1
    
    failure_fixed_count.append(fixed_count)

sum(failure_fixed_count)/len(failure_fixed_count)

In [ ]:
import csv

with open('win-7-succs.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["successes"])

    for x in success_fixed_count:
        writer.writerow([x])


In [ ]:
import csv

with open('win-7-fails.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["failures"])

    for x in failure_fixed_count:
        writer.writerow([x])

In [ ]:
from importlib import import_module

In [ ]:
import_module("nbformat")

In [ ]:
import nbformat

In [ ]:
!pip uninstall rpds-py

In [ ]:
!pip show nbformat

In [ ]:
!pip install --upgrade notebook jupyter jupyterlab

In [ ]:
!pip install --upgrade plotly

In [ ]:
00110100

In [ ]:
convert_tokens_to_perm([12,13,14,15,16,17])

In [ ]:
k = 0
seq = val_seqs[k]
perm = val_perms[k]
perm

In [ ]:
a = model.module.generate(seq)

In [ ]:
# Calculate and print accuracy after each epoch
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode

    # calculate validation stats
    total_accuracy = 0.0
    total_loss = 0.0

    num_batches = 0

    print("Evaluating...")
    for inputs, targets in tqdm(val_dataloader):
        outputs = model(inputs)

        0/0

        # calculate the val accuracy
        accuracy = calculate_accuracy(outputs, targets)
        total_accuracy += accuracy

        # Calculate the val loss
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        num_batches += 1

    average_accuracy = total_accuracy / num_batches
    val_loss = total_loss / num_batches

    metrics = {
        "validation_accuracy": average_accuracy,
        "loss": val_loss,
        "training_accuracy": average_train_accuracy,
        "training_loss": train_loss,
    }

In [ ]:
inputs[2]

In [ ]:
outputs[2]

In [ ]:
torch.argmax(outputs[2])

In [ ]:
model.module.softmax(outputs)[1]

In [ ]:
targets[1]

In [ ]:
val_perms[0]

In [ ]:
import pyperclip

def np_to_mathematica(array, copy=True):
    formatted = str(array.tolist()).replace("[", "{").replace("]", "}")
    return formatted

In [ ]:
pyperclip.copy(np_to_mathematica(embedding_pca))
print("Copied!")

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

embedding = np.array(model.module.token_embedding_table.weight.cpu().detach().numpy())
pos_embedding = np.array(model.module.position_embedding.weight.cpu().detach().numpy())

pca = PCA(n_components=3)
pca.fit(embedding)

embedding_pca = pca.transform(embedding)

pca = PCA(n_components=2)
pca.fit(pos_embedding)

pos_embedding_pca = pca.transform(pos_embedding)

In [ ]:
import plotly.express as px

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y))
    similarity.append(row)

px.imshow(similarity)

In [ ]:
np.dot(pos_embedding[MAX_LENGTH], embedding[START_PREDICTION_TOKEN])

In [ ]:
convert_to_transposition(13)

In [ ]:
threshold = 30

for pos1, x in enumerate(embedding):
    for pos2, y in enumerate(embedding):
        if np.dot(x, y) > threshold and pos1 != pos2:
            print("x", pos1, "y", pos2, "dot", np.dot(x, y))

In [ ]:
px.imshow(model.module.token_embedding_table.weight.detach())

In [ ]:
torch.no_grad()
model.eval()

In [ ]:
targets

In [ ]:
# calculate validation stats
criterion = nn.CrossEntropyLoss()

total_accuracy = 0.0
total_loss = 0.0

num_batches = 0

print("Evaluating...")
for inputs, targets in tqdm(val_dataloader):
    outputs = model(inputs)

    # calculate the val accuracy
    accuracy = calculate_accuracy(outputs, targets)
    total_accuracy += accuracy

    # Calculate the val loss
    loss = criterion(outputs, targets)
    total_loss += loss.item()
    num_batches += 1

average_accuracy = total_accuracy / num_batches
val_loss = total_loss / num_batches

In [ ]:
val_loss

In [ ]:
vocab_size

In [ ]:
output = model(data)

In [ ]:
train

In [ ]:
train.shape

In [ ]:
output.shape

In [ ]:
output[1]

In [ ]:
model.get_device()

In [ ]:
dev

In [ ]:
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

In [ ]:
conver

In [ ]:
model.eval()

# use gpu for processing
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

# create an initial input
input_tensor = torch.ones(block_size, dtype=int).to(dev)
input_tensor *= TO_PREDICT_TOKEN
input_tensor[:len(seq)] = torch.tensor(seq, dtype=int).to(dev)
input_tensor[len(seq)] = START_PREDICTION_TOKEN

In [ ]:
input_tensor.unsqueeze(0)

In [ ]:
model(input_tensor.unsqueeze(0))

In [ ]:
torch.argmax(model(input_tensor.unsqueeze(0)), dim=1)

In [ ]:
prediction_tensor = torch.zeros(block_size, dtype=int).to(dev)

In [ ]:
sequence = [1,2,3,1,2,3]

In [ ]:
prediction_tensor[:len(sequence)] = torch.tensor(sequence, dtype=int).to(dev)

In [ ]:
prediction_tensor

In [ ]:
input_tensor = torch.ones(block_size, dtype=int).to(dev)

In [ ]:
input_tensor *= TO_PREDICT_TOKEN

In [ ]:
input_tensor

## pretty pictures

In [ ]:
with open("./embedding_pictures/position/torn-1.0.npy", "rb") as file:
    pos_pictures = np.load(file)

with open("./embedding_pictures/token/torn-1.0.npy", "rb") as file:
    token_pictures = np.load(file)

In [ ]:
def show_image(image):
    return px.imshow(
        image, 
        animation_frame=0, 
        zmin=image.min(),
        zmax=image.max()
    )

show_image(token_pictures)